## Data source

https://lpdaac.usgs.gov/products/nasadem_hgtv001/

## Setup

#### Imports

In [ ]:
# Data tools
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime

# Datacube
import datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py
from datacube.utils.rio import configure_s3_access

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

# Python
import sys, os, re

# Optional EASI tools
sys.path.append(os.path.expanduser('~/hub-notebooks/scripts'))
import notebook_utils

### Dask

In [ ]:
cluster, client = notebook_utils.initialize_dask(workers=(1,2), use_gateway=False, wait=False)
display(cluster if cluster else client)
notebook_utils.localcluster_dashboard(client)

### AWS configuration

In [ ]:
"""This function obtains credentials for S3 access and passes them on to
   processing threads, either local or on dask cluster.
   Note that AWS credentials may need to be renewed between sessions or
   after a period of time."""

os.environ["AWS_HTTPS"] = "NO"
os.environ["GDAL_HTTP_PROXY"] = "easi-caching-proxy.caching-proxy:80"

from datacube.utils.aws import configure_s3_access
configure_s3_access(
    aws_unsigned=False,
    requester_pays=True,
    AWS_HTTPS=os.environ["AWS_HTTPS"],
    GDAL_HTTP_PROXY=os.environ["GDAL_HTTP_PROXY"],
    client=client
)

In [ ]:
dc = datacube.Datacube()

In [ ]:
arcsec = 1./3600

### Area of interest (Lake Tempe)
latitude = (-4.2, -3.9)
longitude = (119.8, 120.1)

query = {
    "product": "lpdaac_nasadem",
    "x": longitude,
    "y": latitude,
    "crs": 'epsg:4326',
    "output_crs": 'epsg:4326',
    "resolution": (-arcsec, arcsec),
    "dask_chunks": {"latitude":2048, "longitude":2048},
}

In [ ]:
x = dc.load(**query)
display(x)

In [ ]:
import hvplot.xarray

options = {
    'title': 'NASA DEM',
    'width': 800,
    'height': 800,
    'aspect': 'equal',
    'cmap': 'seismic',
    'clim': (-5,100),    # Limit the color range depending on the layer_name
    'colorbar': True,
    'tools': ['hover'],
}

x.hvplot.image(
     x = 'longitude', y = 'latitude',         # Dataset x,y dimension names 
     rasterize = True,                        # If False, data will not be reduced. This is slow to load but all data is loaded.
     precompute = True,                       # Datashader precomputes what it can
    ).opts(**options).hist(bin_range = options['clim'])